This notebook is meant to give basic idea of how pedigree files were provided and what kinf of information they have. In addition, here is presented a basic statistics of raw files as well as highlighted some issues that could help in constructing the script for reformatting and merging 2 pedigrees.

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns

/tmp/ipykernel_674742/523115141.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [4]:
sheet_to_df_map = pd.read_excel("~/projects/thb_research/pedigree_analysis/data/raw_pedigree_part1.xlsx", 
                                sheet_name=None, 
                                dtype=str,
                                na_values=[" ", "", "None"]) # just to unify missing values

# the names of the sheets are:
# PedIDMatch (matching suffixes of genome id written in plink files with ped id), 
# PedNew (pedigree phenotypes info), 
# GenotypeIDs (info about genotypes)

pedid_match = sheet_to_df_map["PedIDMatch"]
ped_df = sheet_to_df_map["PedNew"]
geno_id = sheet_to_df_map["GenotypeIDs"]

# read a pedigree with additional 6k data without info on pedigree
ped_addit = pd.read_csv("~/projects/thb_research/pedigree_analysis/data/raw_pedigree_part2.csv", 
                        dtype=str, na_values=[" ", "", "None"], 
                        usecols=range(12)).query("id.notna().values") # to read only those rows that has an actual records

In [ ]:
# PedIdMatch
pedid_match.head()

In [15]:
pedid_match["Equinome ID"].nunique()

14695

In [16]:
pedid_match["horse_id"].nunique()

13877

In [14]:
pedid_match.shape

(14695, 3)

In [28]:
pedid_match.groupby("horse_id", as_index=False).agg({"Equinome ID": "count"}).query("`Equinome ID` > 1").shape

(762, 2)

In [27]:
pedid_match.groupby("horse_id", as_index=False).agg({"Equinome ID": "count"}).query("`Equinome ID` > 2").shape

(56, 2)

In [29]:
pedid_match.groupby("horse_id", as_index=False).agg({"Equinome ID": "count"}).query("`Equinome ID` > 3").shape

(0, 2)

In [9]:
pedid_match.shape

(14695, 3)

In [30]:
pedid_match.isna().any()

Equinome ID    False
Horse Name     False
horse_id       False
dtype: bool

In [6]:
# PedNew
ped_df.head()

,id,status,name,sire_id,dam_id,YOB,sex,colour,COB
0,7,0,battle joined,3,6,1959,1,b.,USA
1,177841,0,armed venus,3,452020,1958,2,b.,USA
2,448331,0,gentility,3,448330,1961,2,b.,USA
3,15,0,ack ack,7,14,1966,1,b.,USA
4,383725,0,jungle war,7,383724,1964,2,b.,USA


In [10]:
ped_df.shape

(68655, 9)

In [31]:
ped_df.id.nunique()

68655

In [59]:
ped_df[ped_df.duplicated(["name", "dam_id", "YOB"], keep=False)]

,id,status,name,sire_id,dam_id,YOB,sex,colour,COB
56785,196235,3,abadan,NaN,NaN,NaN,1,NaN,GB
56792,265453,3,abadan,NaN,NaN,NaN,1,NaN,GB
56796,385381,3,abbots trace,NaN,NaN,NaN,1,NaN,IRE
56797,455953,3,abbots trace,NaN,NaN,NaN,1,NaN,IRE
56836,473013,3,abbots trace,NaN,NaN,NaN,1,NaN,IRE
...,...,...,...,...,...,...,...,...,...
68630,385541,3,zingara,NaN,NaN,NaN,2,NaN,FR
68631,444915,3,zingara,NaN,NaN,NaN,2,NaN,FR
68650,190775,3,zvornik,NaN,NaN,NaN,1,NaN,AUS
68651,295979,3,zvornik,NaN,NaN,NaN,1,NaN,AUS


In [32]:
ped_df.isna().sum()

id             0
status         0
name           2
sire_id    11871
dam_id     11871
YOB         9927
sex         1944
colour     14813
COB         4032
dtype: int64

In [38]:
ped_df.apply(lambda x: ((1-(x.count()/x.size))*100).round(2)) 

id          0.00
status      0.00
name        0.00
sire_id    17.29
dam_id     17.29
YOB        14.46
sex         2.83
colour     21.58
COB         5.87
dtype: float64

In [7]:
geno_id.head()

,id,batchID,equinomeID,SNPChip,Year of Birth,Country Reported,sex
0,20160112_JSB001_409,20160112,JSB001_409,SNP670,1971,Ireland,Female
1,20141029_CM108,20141029,CM108,SNP70,1973,Ireland,Female
2,20100101_CM002,20100101,CM002,SNP50,1974,Ireland,Male
3,20100101_CM104,20100101,CM104,SNP50,1975,Ireland,Female
4,20141029_CM110,20141029,CM110,SNP70,1976,Ireland,Female


In [11]:
geno_id.shape

(11183, 7)

In [61]:
geno_id.id.nunique()

11183

In [62]:
geno_id.equinomeID.nunique()

11107

In [5]:
ped_addit.head()

,id,batchID,equinomeID,SNPChip,Year of Birth,sex,Country Reported,Horse Name,Sire,Dam,Month of Birth,Country of Birth
0,20100101_103,20100101,103,SNP50,2003,Male,New Zealand,Condesire,Traditionally,Fine Feather,September,New Zealand
1,20100101_GB5,20100101,GB5,SNP50,1999,Female,Ireland,Susan Glen,Sacrament,SanvacGlen,May,Ireland
2,20100101_GF01,20100101,GF01,SNP50,2000,Male,Ireland,Rossmore Castle,WitnessBox,LatinQuarter,June,Ireland
3,20100101_GF03,20100101,GF03,SNP50,2003,Male,Ireland,Radiator Rooney,Elnadim,QueenofTheMay,April,Ireland
4,20100101_JSB049,20100101,JSB049,SNP50,2004,Female,Ireland,Ambereen,Lils Boy,Aeraiocht,February,Ireland


In [7]:
ped_addit.columns

Index(['id', 'batchID', 'equinomeID', 'SNPChip', 'Year of Birth', 'sex',
       'Country Reported', 'Horse Name', 'Sire', 'Dam', 'Month of Birth',
       'Country of Birth'],
      dtype='object')

In [6]:
ped_addit.shape

(5814, 12)

In [8]:
ped_addit[ped_addit.duplicated("id")]

,id,batchID,equinomeID,SNPChip,Year of Birth,sex,Country Reported,Horse Name,Sire,Dam,Month of Birth,Country of Birth


In [12]:
ped_addit[ped_addit.duplicated("equinomeID", keep=False)] \
    .sort_values("equinomeID")

,id,batchID,equinomeID,SNPChip,Year of Birth,sex,Country Reported,Horse Name,Sire,Dam,Month of Birth,Country of Birth
83,20141020_BAR001_021,20141020,BAR001_021,SNP70,2012,Male,Australia,Yum Sing,Congrats,Hennessy Belle,October,Australia
1605,20160627_BAR001_021,20160627,BAR001_021,SNP70,2012,Male,Australia,Yum Sing,Congrats,Hennessy Belle,October,Australia
1608,20160627_BAR001_032,20160627,BAR001_032,SNP70,2012,Male,Australia,Congratulations,Congrats,Double Strength,September,Australia
410,20150105_BAR001_032,20150105,BAR001_032,SNP70,2012,Male,Australia,Congratulations,Congrats,Double Strength,September,Australia
2103,20170214_CUM001_117,20170214,CUM001_117,SNP70_PVL,2013,Female,Great Britain,Angela North,Canford Cliffs,Vallota,April,Great Britain
...,...,...,...,...,...,...,...,...,...,...,...,...
838,20150813_SFI152,20150813,SFI152,SNP670,2012,Male,Ireland,Family Pride,Proud Citizen,Family,March,Ireland
427,20150105_TLS001_002,20150105,TLS001_002,SNP70,2012,Female,Australia,Noaf,Commands,Crossyourheart,September,Australia
1601,20160614_TLS001_002,20160614,TLS001_002,SNP70,2012,Female,Australia,Noaf,Commands,Crossyourheart,September,Australia
429,20150105_WPS001_010,20150105,WPS001_010,SNP70,2012,Male,Australia,Astro Fame,Encosta de Lago,Special Episode,September,Australia


In [13]:
ped_df.head()

,id,status,name,sire_id,dam_id,YOB,sex,colour,COB
0,7,0,battle joined,3,6,1959,1,b.,USA
1,177841,0,armed venus,3,452020,1958,2,b.,USA
2,448331,0,gentility,3,448330,1961,2,b.,USA
3,15,0,ack ack,7,14,1966,1,b.,USA
4,383725,0,jungle war,7,383724,1964,2,b.,USA
